# Audio compression with EnCodec and OpenVINO

Compression is an important part of the Internet today because it enables people to easily share high-quality photos, listen to audio messages, stream their favorite shows, and so much more. Even when using today’s state-of-the-art techniques, enjoying these rich multimedia experiences requires a high speed Internet connection and plenty of storage space. AI helps to overcome these limitations: "Imagine listening to a friend’s audio message in an area with low connectivity and not having it stall or glitch."

This tutorial considers ways to use OpenVINO and EnCodec algorithm for hyper compression of audio.
EnCodec is a real-time, high-fidelity audio codec that uses AI to compress audio files without losing quality. It was introduced in [High Fidelity Neural Audio Compression](https://arxiv.org/pdf/2210.13438.pdf) paper by Meta AI. The researchers claimed they achieved an approximate 10x compression rate without loss of quality and made it work for CD-quality audio. More details about this approach can be found in [Meta AI blog](https://ai.facebook.com/blog/ai-powered-audio-compression-technique/) and original [repo](https://github.com/facebookresearch/encodec).

![image.png](https://github.com/openvinotoolkit/openvino_notebooks/assets/29454499/17546d66-12b9-4841-9293-cc878258a186)


## Prerequisites

Install required dependencies:

In [ ]:
!python -W ignore -m pip install -q -r requirements.txt

## Instantiate audio compression pipeline

[Codecs](https://en.wikipedia.org/wiki/Codec), which act as encoders and decoders for streams of data, help empower most of the audio compression people currently use online. Some examples of commonly used codecs include MP3, Opus, and EVS. Classic codecs like these decompose the signal between different frequencies and encode as efficiently as possible. Most classic codecs leverage human hearing knowledge (psychoacoustics) but have a finite or given set of handcrafted ways to efficiently encode and decode the file. EnCodec, a neural network that is trained from end to end to reconstruct the input signal, was introduced as an attempt to overcome this limitation. It consists of three parts:

* The **encoder**, which takes the uncompressed data in and transforms it into a higher dimensional and lower frame rate representation.

* The **quantizer**, which compresses this representation to the target size. This compressed representation is what is stored on disk or will be sent through the network.

* The **decoder** is the final step. It turns the compressed signal back into a waveform that is as similar as possible to the original. The key to lossless compression is to identify changes that will not be perceivable by humans, as perfect reconstruction is impossible at low bit rates.

![encodec_compression](https://github.com/openvinotoolkit/openvino_notebooks/assets/29454499/5cd9a482-b42b-4dea-85a5-6d66b20ce13d))


The authors provide two multi-bandwidth models:
* `encodec_model_24khz` - a causal model operating at 24 kHz on monophonic audio trained on a variety of audio data.
* `encodec_model_48khz` - a non-causal model operating at 48 kHz on stereophonic audio trained on music-only data.

In this tutorial, we will use `encodec_model_24khz` as an example, but the same actions are also applicable to `encodec_model_48khz` model as well.
To start working with this model, we need to instantiate model class using EncodecModel.encodec_model_24khz() and select required compression bandwidth among available: 1.5, 3, 6, 12 or 24 kbps for 24 kHz model and 3, 6, 12 and 24 kbps for 48 kHz model. We will use 6 kbs bandwidth.

In [ ]:
from encodec import EncodecModel, compress, decompress
from encodec.utils import convert_audio, save_audio
import torchaudio
import torch
import typing as tp

# Instantiate a pretrained EnCodec model
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(6.0)

## Explore EnCodec pipeline

Let us explore model capabilities on example audio:

In [ ]:
import sys
import librosa
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

sys.path.append("../utils")

from notebook_utils import download_file

test_data_url = "https://github.com/facebookresearch/encodec/raw/main/test_24k.wav"

sample_file = 'test_24k.wav'
download_file(test_data_url, sample_file)
audio, sr = librosa.load(sample_file)
plt.figure(figsize=(14, 5))
librosa.display.waveshow(audio, sr=sr)

ipd.Audio(sample_file)

### Preprocessing

To achieve the best result, audio should have the number of channels and sample rate expected by the model. If audio does not fulfill these requirements, it can be converted to the desired sample rate and the number of channels using the `convert_audio` function.

In [ ]:
model_sr, model_channels = model.sample_rate, model.channels
print(f"Model expected sample rate {model_sr}")
print(f"Model expected audio format {'mono' if model_channels == 1 else 'stereo'}")

In [ ]:
# Load and pre-process the audio waveform
wav, sr = torchaudio.load(sample_file)

wav = convert_audio(wav, sr, model_sr, model_channels)

### Encoding

Audio waveform should be split by chunks and then encoded by Encoder model, then compressed by quantizer for reducing memory. The result of compression is a binary file with `ecdc` extension, a special format for storing EnCodec compressed audio on disc.

In [ ]:
from pathlib import Path


out_file = Path("compressed.ecdc")
b = compress(model, wav)
out_file.write_bytes(b)

Let us compare obtained compression result:

In [ ]:
import os

orig_file_stats = os.stat(sample_file)
compressed_file_stats = os.stat("compressed.ecdc")
print(f"size before compression in Bytes: {orig_file_stats.st_size}")
print(f"size after compression in Bytes: {compressed_file_stats.st_size}")
print(f"Compression file size ratio: {orig_file_stats.st_size / compressed_file_stats.st_size:.2f}")

Great! Now, we see the power of hyper compression. Binary size of a file becomes 60 times smaller and more suitable for sending via network.

### Decompression

After successful sending of the compressed audio, it should be decompressed on the recipient's side. The decoder model is responsible for restoring the compressed signal back into a waveform that is as similar as possible to the original.

In [ ]:
out, out_sr = decompress(out_file.read_bytes())

In [ ]:
output_file = "decopressed.wav"
save_audio(out, output_file, out_sr)

The decompressed audio will be saved to the `decompressed.wav` file when decompression is finished. We can compare result with the original audio.

In [ ]:
audio, sr = librosa.load(output_file)
plt.figure(figsize=(14, 5))
librosa.display.waveshow(audio, sr=sr)

ipd.Audio(output_file)

Nice! Audio sounds close to original.

## Convert model to OpenVINO Intermediate Representation format

For best results with OpenVINO, it is recommended to convert the model to OpenVINO IR format. OpenVINO supports PyTorch via ONNX conversion. We will use `torch.onnx.export` for exporting the ONNX model from PyTorch. We need to provide initialized model's instance and example of inputs for shape inference. We will use `mo.convert_model` functionality to convert the ONNX models. The `mo.convert_model` Python function returns an OpenVINO model ready to load on the device and start making predictions. We can save it on disk for the next usage with `openvino.runtime.serialize`.

In [ ]:
class FrameEncoder(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x: torch.Tensor):
        codes, scale = self.model._encode_frame(x)
        if not self.model.normalize:
            return codes
        return codes, scale        

In [ ]:
class FrameDecoder(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, codes, scale=None):
        return model._decode_frame((codes, scale))

In [ ]:
encoder = FrameEncoder(model)
decoder = FrameDecoder(model)

In [ ]:
from openvino.tools import mo
from openvino.runtime import Core, serialize

core = Core()

OV_ENCODER_PATH = Path("encodec_encoder.xml")
if not OV_ENCODER_PATH.exists():
    torch.onnx.export(encoder, torch.zeros(1, 1, 480000), "encodec_encoder.onnx")
    encoder_ov = mo.convert_model("encodec_encoder.onnx", compress_to_fp16=True)
    serialize(encoder_ov, str(OV_ENCODER_PATH))
else:
    encoder_ov = core.read_model(OV_ENCODER_PATH)

In [ ]:
OV_DECODER_PATH = Path("encodec_decoder.xml")
if not OV_DECODER_PATH.exists():
    torch.onnx.export(decoder, torch.zeros([1, 8, 1500], dtype=torch.long), "encodec_decoder.onnx", input_names=["codes", "scale"])
    decoder_ov = mo.convert_model("encodec_decoder.onnx", compress_to_fp16=True)
    serialize(decoder_ov, str(OV_DECODER_PATH))
else:
    decoder_ov = core.read_model(OV_DECODER_PATH)

## Integrate OpenVINO to EnCodec pipeline

The following steps are required for integration of OpenVINO to EnCodec pipeline:

1. Load the model to a device.
2. Define audio frame processing functions.
3. Replace the original frame processing functions with OpenVINO based algorithms.

In [ ]:
device = "CPU"

compiled_encoder = core.compile_model(encoder_ov, device)
encoder_out = compiled_encoder.output(0)

compiled_decoder = core.compile_model(decoder_ov, device)
decoder_out = compiled_decoder.output(0)

In [ ]:
def encode_frame(x: torch.Tensor):
    has_scale = len(compiled_encoder.outputs) == 2
    result = compiled_encoder(x)
    codes = torch.from_numpy(result[encoder_out])
    if has_scale:
        scale = torch.from_numpy(result[compiled_encoder.output(1)])
    else:
        scale = None
    return codes, scale

In [ ]:
EncodedFrame = tp.Tuple[torch.Tensor, tp.Optional[torch.Tensor]]


def decode_frame(encoded_frame: EncodedFrame):
    codes, scale = encoded_frame
    inputs = [codes]
    if scale is not None:
        inputs.append(scale)
    return torch.from_numpy(compiled_decoder(inputs)[decoder_out])

In [ ]:
model._encode_frame = encode_frame
model._decode_frame = decode_frame

## Run EnCodec with OpenVINO

The process of running encodec with OpenVINO under hood will be the same like with the original PyTorch models.

In [ ]:
b = compress(model, wav, use_lm=False)
out_file = Path("compressed_ov.ecdc")
out_file.write_bytes(b)

In [ ]:
out, out_sr = decompress(out_file.read_bytes())

In [ ]:
ov_output_file = "decopressed_ov.wav"
save_audio(out, ov_output_file, out_sr)

In [ ]:
audio, sr = librosa.load(ov_output_file)
plt.figure(figsize=(14, 5))
librosa.display.waveshow(audio, sr=sr)

ipd.Audio(ov_output_file)

In [ ]:
import gradio as gr
from socket import gethostname, gethostbyname
from typing import Tuple
import numpy as np

def preprocess(input, sample_rate, model_sr, model_channels):
    input = torch.tensor(input, dtype=torch.float32)
    input = input / 2**15  # adjust to int16 scale
    input = input.unsqueeze(0)
    input = convert_audio(input, sample_rate, model_sr, model_channels)
    
    return input


def postprocess(output):
    output = output.squeeze()
    output = output * 2**15  # adjust to [-1, 1] scale
    output = output.numpy(force=True)
    output = output.astype(np.int16)

    return output
    

def _compress(input: Tuple[int, np.ndarray]):
    sample_rate, waveform = input
    waveform = preprocess(waveform, sample_rate, model_sr, model_channels)
    
    b = compress(model, waveform, use_lm=False)
    out, out_sr = decompress(b)
    
    out = postprocess(out)
    return out_sr, out
    
demo = gr.Interface(
    _compress,
    'audio',
    'audio',
    examples=['test_24k.wav']
)
    
ipaddr = gethostbyname(gethostname())
demo.launch(server_name=ipaddr)